# Generating a Bench Mark Strategy 

# Implementing a buy and hold strategy to simulate most investors in the market who usually hold historically reliable stock on the S&P500 such as the ones we are using, to show that our strategiy can outperform this

In [16]:
# loading in libraries 

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [17]:
# loading in stock data 

raw = pd.read_csv('Trading_Project_Data.csv', index_col = 0, parse_dates = True, header = [0,1])

names_full = ['AAPL','AMZN', 'BA', 'CAT', 'CVX', 'GOOGL', 'GS', 'JNJ', 'JPM', 'KO','MCD', 'MSFT','NKE','NVDA','PFE','SOFI', 'TSLA', 'UNH', 'WMT','XOM']

names = ['AMZN','CAT','GS', 'JNJ','JPM','NKE','NVDA','SOFI','WMT', 'XOM']


/var/folders/ww/t2gpxbpj1dxghvgyxhxsyczm0000gn/T/ipykernel_85543/1255623039.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  raw = pd.read_csv('Trading_Project_Data.csv', index_col = 0, parse_dates = True, header = [0,1])


In [18]:
stock_eod = pd.DataFrame(raw['Close']) # Gathering Closing Price data 
stock_eod = stock_eod[names] # extracting the data for the teams 10 selected stocks 

dates = stock_eod.index
index = dates.get_loc('2024-03-01') # Cutting off data to only trade on specified "Trading period Dates"
stock_eod_bench = stock_eod[index:]

cols = stock_eod_bench[names]
log_cols = []

# Generating Log Returns for the 10 specific stocks 
for col in cols:
    stock_eod_bench[f'log_ret_{col}'] = np.log(stock_eod_bench[col]) - np.log(stock_eod_bench[col].shift(1))
    log_cols.append(f'log_ret_{col}')


stock_eod_benchmark = stock_eod_bench.dropna()


/var/folders/ww/t2gpxbpj1dxghvgyxhxsyczm0000gn/T/ipykernel_85543/779382411.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_eod_bench[f'log_ret_{col}'] = np.log(stock_eod_bench[col]) - np.log(stock_eod_bench[col].shift(1))
/var/folders/ww/t2gpxbpj1dxghvgyxhxsyczm0000gn/T/ipykernel_85543/779382411.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_eod_bench[f'log_ret_{col}'] = np.log(stock_eod_bench[col]) - np.log(stock_eod_bench[col].shift(1))
/var/folders/ww/t2gpxbpj1dxghvgyxhxsyczm0000gn

In [19]:
# A function which takes in the log returns and wealth and outputs a column showing the amount made over trading period by utilising 
# benchmar strategy until final trading day 
def buy_and_hold_benchmark(wealth: float, df: pd.DataFrame, log_ret_stock: str) -> pd.Series:
    if log_ret_stock not in df.columns:
        raise ValueError(f"Stock '{log_ret_stock}' not found in DataFrame columns.")

    cum_log_returns = df[log_ret_stock].cumsum()
    investment_amount = wealth / 10
    benchmark = investment_amount * np.exp(cum_log_returns)

    return benchmark

# Constants
wealth = 50000
log_return_columns = [
    "log_ret_AMZN", "log_ret_CAT", "log_ret_GS", "log_ret_JNJ", "log_ret_JPM",
    "log_ret_NKE", "log_ret_NVDA", "log_ret_SOFI", "log_ret_WMT", "log_ret_XOM"
]

# Create the output DataFrame
benchmark_df = pd.DataFrame(index=stock_eod_benchmark.index)

for stock_col in log_return_columns:
    benchmark_df[stock_col] = buy_and_hold_benchmark(wealth, stock_eod_benchmark, stock_col)


In [20]:
column_mapping = {
    'log_ret_AMZN': 'AMZN Benchmark',
    'log_ret_CAT': 'CAT Benchmark',
    'log_ret_GS': 'GS Benchmark',
    'log_ret_JNJ': 'JNJ Benchmark',
    'log_ret_JPM': 'JPM Benchmark',
    'log_ret_NKE': 'NKE Benchmark',
    'log_ret_NVDA': 'NVDA Benchmark',
    'log_ret_SOFI': 'SOFI Benchmark',
    'log_ret_WMT': 'WMT Benchmark',
    'log_ret_XOM': 'XOM Benchmark'
}

# Apply the renaming using the mapping
benchmark_df = benchmark_df.rename(columns=column_mapping)
benchmark_df

,AMZN Benchmark,CAT Benchmark,GS Benchmark,JNJ Benchmark,JPM Benchmark,NKE Benchmark,NVDA Benchmark,SOFI Benchmark,WMT Benchmark,XOM Benchmark
Date,,,,,,,,,,
2024-03-04,4982.044681,5028.956826,5053.465663,4929.681420,5037.509135,4885.159128,5179.754218,4771.968934,5045.949915,4930.083527
2024-03-05,4884.973458,4952.925146,5036.201843,4933.691308,5087.970419,4824.793982,5224.178345,4043.381627,5108.917612,4990.552244
2024-03-06,4867.859761,4984.110730,5018.938422,4914.261031,5114.415916,4791.421353,5390.449640,4115.684251,5154.016408,5043.934412
2024-03-07,4960.722874,5052.271607,5004.251279,4899.765296,5069.621012,4818.904940,5631.652187,4154.616229,5136.147179,5072.279146
2024-03-08,4919.481678,5036.975753,4985.698719,4919.812813,5079.066106,4866.509863,5319.225693,4288.098017,5115.725250,5119.992805
...,...,...,...,...,...,...,...,...,...,...
2025-01-10,6142.408285,5271.532119,7337.405796,4485.970448,6620.795964,3541.594214,8261.436162,7869.855384,8008.399897,5156.828480
2025-01-13,6128.941904,5444.246331,7376.058361,4562.073539,6740.587234,3585.366972,8098.528986,7825.361540,7881.815407,5289.936028
2025-01-14,6109.302913,5580.465044,7488.478124,4570.915341,6830.568302,3540.102030,8009.173377,8064.516334,7818.092839,5310.749223


In [21]:
# Summing each of the stocks final positions after buying and holding over whole period to find final portfolio position 
total_portfolio_value = benchmark_df.iloc[-1].sum()

In [23]:
# Final bench mark strategy portfolio value
total_portfolio_value

65594.38956734147